In [1]:
from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd
import scraperwiki
from string import ascii_lowercase
import urllib2
import urllib
import time

In [37]:
url = 'http://www.fightmetric.com/statistics/fighters'
html = scraperwiki.scrape(url)
soup = BeautifulSoup(html, 'lxml')

In [38]:
table_fighter = soup.find_all('tbody')[0]
rows_fighter = table_fighter.find_all('tr')[1:]

In [39]:
def get_career_stats(url):
    ret = []
    html = scraperwiki.scrape(url)
    soup = BeautifulSoup(html, 'lxml')
    div_stats = soup.find_all('div', {'class': 'b-list__info-box-left clearfix'})[0]
    list_items_stats = div_stats.find_all('li', {'class': 'b-list__box-list-item b-list__box-list-item_type_block'})
    for i in list_items_stats:
        txt = i.text.strip()
        txt = ' '.join(txt.split()).split(': ')[-1]
        if len(txt) > 0:
#             print (txt)
            ret.append(txt)
    return ret

In [40]:
test_url = 'http://www.fightmetric.com/fighter-details/b361180739bed4b0'
get_career_stats(test_url)

[u'1.35', u'30%', u'3.55', u'38%', u'1.07', u'33%', u'66%', u'0.0']

In [41]:
#loop through a single fighter row and capture all the information
def parse_row(row):
    ret = []
    career_stats = []
    
    cols_fighter = row.find_all('td')
    for idx, col in enumerate(cols_fighter):
        links = col.find_all('a')
        num_links = len(links)
        if num_links == 1:
            link = links[0]
            try:
                ret.append(link.contents[0])
                if idx == 0:
                    career_stats = get_career_stats(link['href'])
            except IndexError as e:
                ret.append('n/a')
        elif num_links > 1:
            raise Exception('invalid link parsing')
        else:
    #         print repr(col.contents[0])
            txt = col.contents[0].strip()
            if txt != '':
                if '-' in txt:
                    ret.append('n/a')
                else:
                    ret.append(col.contents[0].strip())
            else:
                ret.append('n/a')
    ret = ret + career_stats
#     print ret
    return ret

In [ ]:
header = ['first', 'last', 'nickname', 'height', 'weight', 'reach', 
          'stance', 'w', 'l', 'd', 'belt', 'SLpM', 'Str. Acc.', 'SApM',
         'Str. Def', 'TD Avg', 'TD Acc.', 'TD Def.', 'Sub. Avg.']

In [ ]:
res = []
for c in ascii_lowercase[15:16]:
    url = 'http://www.fightmetric.com/statistics/fighters?char=%c&page=all' %c
    html = scraperwiki.scrape(url)
    soup = BeautifulSoup(html, 'lxml')
    table_fighter = soup.find_all('tbody')[0]
    rows_fighter = table_fighter.find_all('tr')[1:]
    for row in rows_fighter:
        try:
            res.append(parse_row(row))
        except:
            print 'there was an issue with this url: %s' %url

there was an issue with this url: http://www.fightmetric.com/statistics/fighters?char=p&page=all
there was an issue with this url: http://www.fightmetric.com/statistics/fighters?char=p&page=all


In [ ]:
df = pd.DataFrame(res, columns=header)
df.to_csv('tmp_data.csv', index=False)

In [2]:
#pull all of Bisping's fights
url = 'http://www.fightmetric.com/fighter-details/2b93ebd9f5417ad2'
html = scraperwiki.scrape(url)
soup = BeautifulSoup(html, 'lxml')

In [17]:
table_fighter = soup.find_all('tbody')[0]
rows_fighter = table_fighter.find_all('a', {'class': 'b-link b-link_style_black'})
rows_fighter = [x for x in rows_fighter if 'fighter' in x['href'] and
                'Bisping' not in x.text.strip()]

rows_results = table_fighter.find_all('i', {'class': 'b-flag__text'})

res = [['name', 'result']]
for idx, row in enumerate(rows_fighter):
    fighter = rows_fighter[idx].text.strip()
    result = rows_results[idx].text.strip()
    res.append([fighter, result])

In [19]:
res_np = np.asarray(res)
np.savetxt('bisping_results.csv', res_np, delimiter=',', fmt='%s')